# Create a forge and execute an evolution cycle
In this notebook, we will go through the very few steps needed to run a forge cycle for a given budget. 
Before running this notebook, you must set up a few things:

**Your access to LLM's providers:**

To try Ebiose, you must have access to at least one LLM provider. To provide the credentials, copy-paste the [`model_endpoints_template.yml`](./../model_endpoints_template.yml) file, rename it as `model_endpoints.yml` and fill it with your own credentials.

**Add the root directory to your Python path**

Depending on your settings, you may need to add the root of the repository to your `PYTHONPATH` environment variable. You may also use a `.env` file to do so. copy-paste the [`.env.template`](./../.env.template) file, rename it as `.env` and fill it with your own root directory.

**Use LangFuse for tracing**

Ebiose has chosen LangFuse to provide easy and free observability, through its self-hosted capability. Refer to [Langfuse official documentation](https://langfuse.com/self-hosting) to set it up. Once done, fill `LANGFUSE_SECRET_KEY`, `LANGFUSE_PUBLIC_KEY` and `LANGFUSE_HOST` in the `.env` file.

Other observability tools might be used but are not configured yet.

**Load .env file**

To load the `.env` file, execute:

In [ ]:
from dotenv import load_dotenv
load_dotenv()

## Creating a basic forge

In ebiose, a **forge** is where custom agents are created to solve specific problems. The forge is the exclusive origin of new agents. Within each forge, architects agents orchestrate the creation and improvement of agents by reusing existing building blocks from the ecosystem.

To create a forge and run a cycle, you must provide the following:
- a description of the forge, which defines the problem that must be solved by generated agents;
- the expected format of the agent's input and output, defined as Pydantic models;
- an implementation of the `compute_fitness` abstract method that will be used by the forge to evaluate the generated agents.

Let's say we wich to generate agents specialized in solving math problems. The forge description could be:

In [3]:
forge_description = "Solving math word problems"

Next, we need to define the expected input and output formats of the generated agents. These formats are to be defined as Pydantic models. 

For instance, in our context of solving math problems, we want the agent input to be a string which will represent the math problem to be solved and the agent output to be composed of two fields:
- `solution` which will be the final solution to the math problem, given as an integer;
- `rationale` which will be the rationale behind the found solution.

The IO Pydantic models will thus be:

In [4]:
from pydantic import BaseModel

class AgentInput(BaseModel):
        math_problem: str

class AgentOutput(BaseModel):
    solution: int
    rationale: str

Lastly, we must provide a way of evaluating the generated agents through the implementation of the `compute_fitness` abstract method of `AgentForge` class. For the sake of demonstration, we will here return a random float between 0 and 1, so that we don't spend tokens at evaluation.

In [5]:
import random
random.seed(7)

from ebiose.core.agent import Agent
from ebiose.core.agent_forge import AgentForge

class BasicForge(AgentForge):
    async def compute_fitness(self, agent: Agent, compute_token_id: str, **kwargs: dict[str, any]) -> float:
        return random.random()

/Users/xabier/dev/ebiose-core/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


We can now instantiate the forge with the provided elements:

In [6]:
forge = BasicForge(
    name="Basic forge",
    description=forge_description,
    agent_input_model=AgentInput,
    agent_output_model=AgentOutput,
    default_generated_agent_engine_type="langgraph_engine",
    default_model_endpoint_id="azure-gpt-4o-mini"
)

# Running a forge cycle

Once the forge is instantiated, we can start generating agents by running a **forge cycle**. 

To do so, you must define:
- a budget in dollars (the forge cycle will end once this budget is exhausted);
- optionally, a path in which created agents and fitness will be saved accross generations. 

> ⚠️ Note that we need to use `asyncio.run` to launch the forge cycle.

> 🚨 Before executing the following cell, check the amount of budget you have allocated!

> 💡 If you are using VSCode, install the [*Markdown Preview Mermaid Support* extension](https://marketplace.visualstudio.com/items?itemName=bierner.markdown-mermaid) to allow the display of the generated agent's graphs.

In [10]:
import asyncio
import nest_asyncio

from ebiose.core.evo_forging_cycle import EvoForgingCylceConfig
nest_asyncio.apply()

from pathlib import Path
from datetime import UTC, datetime

# the path where results will be saved
current_time = datetime.now(UTC).strftime("%Y-%m-%d_%H-%M-%S")
SAVE_PATH = Path(f"./../data/") / current_time
if not SAVE_PATH.exists():
    SAVE_PATH.mkdir(parents=True)

# budget for the forge cycle, in dollars
BUDGET = 0.01

cycle_config = EvoForgingCylceConfig(
    budget=BUDGET,
    n_agents_in_population=2,
    n_selected_agents_from_ecosystem=0,
    replacement_ratio=0.5,
    save_path=SAVE_PATH
)

final_agents, final_fitness = asyncio.run(
    forge.run_new_cycle(config=cycle_config)
)


Starting a new cycle for forge Basic forge
****** Initializing agents population ******
Creating 2 new agents with architect agents...
  0%|          | 0/2 [00:00<?, ?it/s]
Initializing structured output agent for model AgentOutput (1)
 50%|█████     | 1/2 [00:36<00:36, 36.33s/it]BudgetExceededError: Token budget limit exceeded. Limit: 0.01, New total: 0.01022604. Finishing process.
Error when calling azure-gpt-4o-mini: Token budget limit exceeded. Limit: 0.01, New total: 0.01022604
BudgetExceededError: Token budget limit exceeded. Limit: 0.01, New total: 0.011428065000000001. Finishing process.
Error when calling azure-gpt-4o-mini: Token budget limit exceeded. Limit: 0.01, New total: 0.011428065000000001
BudgetExceededError: Token budget limit exceeded. Limit: 0.01, New total: 0.01262976. Finishing process.
Error when calling azure-gpt-4o-mini: Token budget limit exceeded. Limit: 0.01, New total: 0.01262976
BudgetExceededError: Token budget limit exceeded. Limit: 0.01, New total: 0.01

# Agent ID: agent-c7254cbb-8de1-4c02-9c05-3cbbb8dcd67c
## Fitness: 0.7645708662128131
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Problem_Parser(MathProblemParser)
	Math_Problem_Parser(MathProblemParser) --> Operation_Selector(OperationSelector)
	Operation_Selector(OperationSelector) --> Calculation_Executor(CalculationExecutor)
	Calculation_Executor(CalculationExecutor) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network designed to solve math word problems. Each node in this graph has a specific role. The StartNode initiates the process, the LLM nodes perform specialized tasks to analyze and solve the problem, and the EndNode presents the final solution. Your contributions will involve processing information, reasoning through steps, and providing feedback to ensure accurate conclusions. Focus on clarity, precision, and logical reasoning in your outputs.
##### MathProblemParser
You are tasked with interpreting a math word problem. Begin by identifying key components such as numbers, operations (addition, subtraction, multiplication, division), and relationships among them. Break down the problem into smaller parts, and outline the steps needed to solve it. Provide a clear description of each identified component and the overall strategy for solving the problem. If any information is ambiguous or unclear, reflect on it and suggest clarifications or assumptions that could be made. Your output will serve as a foundation for the subsequent steps in the problem-solving process.
##### OperationSelector
Based on the parsed information from MathProblemParser, determine which mathematical operations are required to solve the problem. Analyze the relationships between the identified numbers and decide if you need to perform addition, subtraction, multiplication, or division. Justify your choice of operations, detailing how each operation will contribute to reaching the final solution. Your output will guide the next node in executing the calculations.
##### CalculationExecutor
Using the operations selected by the OperationSelector, perform the calculations needed to solve the math word problem. Clearly show each step of your calculations, ensuring that all arithmetic is correct and follows the order of operations. If you encounter any discrepancies or unusual results, reflect on your calculations and verify their accuracy before proceeding. Your output will provide the numerical answer for the problem.



Saving current state to ../data/2025-03-04_23-36-52/generation=0
Starting crossover and mutation...


ValueError: Sample larger than population or is negative

We can now display the best agents that have been returned as follows. Note that:
- all agents can be found in the `SAVE_PATH` directory if you defined one;
- here, the compute fitness only returns a random float, so the following displayed agents have not been truly evaluated. 

Go check [examples/math_forge/math_forge.py](./../examples/math_forge/math_forge.py) to see a fully implemeted forge with a non-random fitness evaluation function.

In [ ]:
forge.display_results(final_agents, final_fitness)

# Agent ID: agent-69bee958-da80-4f75-9269-6308bb0f1935
## Fitness: 0.7645708662128131
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Understanding the Problem)
	Llm_Node_1(Understanding the Problem) -->|valid interpretation| Llm_Node_2(Solving the Problem)
	Llm_Node_1(Understanding the Problem) -->|invalid interpretation| Llm_Node_1(Understanding the Problem)
	Llm_Node_2(Solving the Problem) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI problem-solving network designed to tackle math word problems. Each node in this network has a specific role: The 'Understanding the Problem' node interprets the problem, identifies key components, and extracts relevant information. The 'Solving the Problem' node applies mathematical reasoning and operations to arrive at a solution. Your task is to work cohesively with the other nodes, ensuring that your outputs are clear, informative, and aligned with the overall goal of providing a solution to the given math word problem. Share insights and be open to feedback to enhance the problem-solving process.
##### Understanding the Problem
You are tasked with interpreting a math word problem. Begin by reading the provided problem carefully. Identify and list the key components, including any numbers, operations, and relationships between elements. Summarize the problem in your own words, highlighting the main question that needs to be answered. Ensure your interpretation is clear and comprehensive, as this will guide the next phase of problem-solving. Provide your insights and be ready to transmit this information to the next node.
##### Solving the Problem
You have received a summary and key components of a math word problem from the 'Understanding the Problem' node. Analyze this information and determine the necessary mathematical operations required to solve the problem. Outline the steps you will take, ensuring to explain your reasoning clearly at each stage. After computing the solution, summarize the final answer concisely, including any relevant details or assumptions made during your calculations. Your output will conclude the problem-solving process, so clarity is essential.

# Agent ID: agent-caa93392-903d-46a5-8f19-09e7caf8e07c
## Fitness: 0.573025940277384
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Understanding the Problem)
	Llm_Node_1(Understanding the Problem) -->|valid interpretation| Llm_Node_2(Solving the Problem)
	Llm_Node_1(Understanding the Problem) -->|invalid interpretation| Llm_Node_3(Feedback Loop)
	Llm_Node_2(Solving the Problem) --> Llm_Node_4(Final Verification)
	Llm_Node_3(Feedback Loop) --> Llm_Node_2(Solving the Problem)
	Llm_Node_4(Final Verification) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI problem-solving network designed to tackle math word problems. Each node in this network has a specific role: The 'Understanding the Problem' node interprets the problem, identifies key components, and extracts relevant information. The 'Solving the Problem' node applies mathematical reasoning and operations to arrive at a solution. The 'Feedback Loop' node provides feedback for refining interpretations, while the 'Final Verification' node ensures the solution's accuracy. Your task is to work cohesively with the other nodes, ensuring that your outputs are clear, informative, and aligned with the overall goal of providing a solution to the given math word problem. Share insights and be open to feedback to enhance the problem-solving process.
##### Understanding the Problem
You are tasked with interpreting a math word problem. Begin by reading the provided problem carefully. Identify and list the key components, including any numbers, operations, and relationships between elements. Summarize the problem in your own words, highlighting the main question that needs to be answered. Ensure your interpretation is clear and comprehensive, as this will guide the next phase of problem-solving. Provide your insights and be ready to transmit this information to the next node.
##### Solving the Problem
You have received a summary and key components of a math word problem from the 'Understanding the Problem' node. Analyze this information and determine the necessary mathematical operations required to solve the problem. Outline the steps you will take, ensuring to explain your reasoning clearly at each stage. After computing the solution, summarize the final answer concisely, including any relevant details or assumptions made during your calculations. Your output will conclude the problem-solving process, so clarity is essential.
##### Feedback Loop
Review the interpretation provided by the 'Understanding the Problem' node. If the interpretation lacks clarity or contains inaccuracies, provide constructive feedback to help refine the interpretation. Indicate whether the interpretation is valid or needs revising, and guide the user on how to address any issues identified.
##### Final Verification
You have received the solution from the 'Solving the Problem' node. Verify the solution's accuracy and completeness. Confirm that all necessary steps were taken and that the final answer is reasonable within the context of the problem. If any discrepancies are found, return to the 'Solving the Problem' node with the necessary adjustments.

